In [9]:
from __future__ import absolute_import, division

import pkg_resources

from builtins import range, str

from addm_toolbox.ddm import DDMTrial, DDM
from addm_toolbox.util import load_trial_conditions_from_csv

In [11]:
d = .006
sigma = .08
range_d = [0.005, 0.006, 0.007]
rangeD = [0.005, 0.006, 0.007]
range_sigma = [0.065, 0.08, 0.095]
rangeSigma = [0.065, 0.08, 0.095]
trials_file_name = None
trialsFileName=None
trials_per_condition = 100
trialsPerCondition=800
num_threads = 9
verbose = False

In [12]:
# Load trial conditions.
if not trialsFileName:
    trialsFileName = pkg_resources.resource_filename(
        u"addm_toolbox", u"test_data/test_trial_conditions.csv")
trialConditions = load_trial_conditions_from_csv(trialsFileName)

# Generate artificial data.
model = DDM(d, sigma)
trials = list()
for (valueLeft, valueRight) in trialConditions:
    for t in range(trialsPerCondition):
        try:
            trials.append(model.simulate_trial(valueLeft, valueRight))
        except:
            print(u"An exception occurred while generating artificial "
                  "trial " + str(t) + u" for condition (" +
                  str(valueLeft) + u", " + str(valueRight) + u").")
            raise

# Get likelihoods for all models and all artificial trials.
numModels = len(rangeD) * len(rangeSigma)
likelihoods = dict()
models = list()
posteriors = dict()
for d in rangeD:
    for sigma in rangeSigma:
        model = DDM(d, sigma)
        if verbose:
            print(u"Computing likelihoods for model " + str(model.params) +
                  u"...")
        try:
            likelihoods[model.params] = model.parallel_get_likelihoods(
                trials, numThreads=numThreads)
        except:
            print(u"An exception occurred during the likelihood "
                  "computations for model " + str(model.params) + u".")
            raise
        models.append(model)
        posteriors[model.params] = 1 / numModels

# Compute the posteriors.
for t in range(len(trials)):
    # Get the denominator for normalizing the posteriors.
    denominator = 0
    for model in models:
        denominator += (posteriors[model.params] *
                        likelihoods[model.params][t])
    if denominator == 0:
        continue

    # Calculate the posteriors after this trial.
    for model in models:
        prior = posteriors[model.params]
        posteriors[model.params] = (likelihoods[model.params][t] *
            prior / denominator)

if verbose:
    for model in models:
        print(u"P" + str(model.params) +  u" = " +
              str(posteriors[model.params]))
    print(u"Sum: " + str(sum(list(posteriors.values()))))

An exception occurred during the likelihood computations for model (0.005, 0.065).


NameError: name 'numThreads' is not defined